In [2]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import sklearn
import sklearn.preprocessing         # For scale function
import sklearn.metrics as metrics    # For for label size

In [3]:
#global data
import importlib
import xpuzzle
importlib.reload(xpuzzle)
import node
importlib.reload(node)
import search
importlib.reload(search)
inputData = np.array([])
goalState=np.array([[1,2,3,4], [5,6,7,0]])

In [22]:
#i/o
#distance = (2-i)+(2-j) if index == -1 else abs(i-(index/self.state.size))+abs(j-(index%self.state.size))
# file.write("\ncost: " + str(moves))
# file.write("\nrunning_time: " + format(time, '.8f'))

def exportData(name, index, data, heu=-1):
    subname = "-h" + str(heu) if heu > -1 else ""
    solution = f"output/{index}_{name}{subname}" + '_solution.txt'
    search = f"output/{index}_{name}{subname}" + '_search.txt'
    file1 = open(solution, 'w')
    file2 = open(search, 'w')
    if isinstance(data[0], node.Node):
        for ele in data[0].solution:
            file1.write(ele)
        file1.write(f"{data[0].gn} {data[2]}")
        for ele in data[1]:
            file2.write(ele)
    else:  
        file1.write(data[0])
        file2.write(data[1])
    file1.close()
    file2.close()
    
def importData(file_name):
    data = []
    file = open(file_name, 'r')
    for line in file:
        data.append([int(x) for x in line.split(" ")])

    data = np.array(data)
    board_len = len(data[0])
    board_side = int(board_len ** 0.5)
    return data

#sample
def generateInput(total, length=8):
    input = []
    for i in range(total):
        input.append(list(np.random.permutation(length)))
    return input

#analysis
def analysis():
    N = 50
    input = generateInput(N)
    ucs_stat = np.zeros(5)
    gbfs0_stat = np.zeros(5)
    gbfs1_stat = np.zeros(5)
    star0_stat = np.zeros(5)
    star1_stat = np.zeros(5)
    for index, e in enumerate(input):
        print(f"***\t\tsample {index}\t\t***")
        t=xpuzzle.XPuzzle(2, 4, input[index]) 
        s=search.Search(t, 15)

        p = s.uniformCost()
        print(p[0])
        if isinstance(p[0], node.Node):
            ucs_stat+=np.array([len(p[0].solution), len(p[1]), p[0].gn, p[2], 1/N])
        p = s.greedyBFS(0)
        if isinstance(p[0], node.Node):
            gbfs0_stat+=np.array([len(p[0].solution), len(p[1]), p[0].gn, p[2], 1/N])
        p = s.greedyBFS(1)
        if isinstance(p[0], node.Node):
            gbfs1_stat+=np.array([len(p[0].solution), len(p[1]), p[0].gn, p[2], 1/N])
        p  = s.aStar(0)
        if isinstance(p[0], node.Node):
            star0_stat+=np.array([len(p[0].solution), len(p[1]), p[0].gn, p[2], 1/N])
        p  = s.aStar(1)
        if isinstance(p[0], node.Node):
            star1_stat+=np.array([len(p[0].solution), len(p[1]), p[0].gn, p[2], 1/N])

    return np.array([ucs_stat, gbfs0_stat, gbfs1_stat, star0_stat, star1_stat])

#scaleup
def scaleup(row, col):
    N = 1
    star1_stat = []
    for r in range(1, row):
        for c in range(3, col):
            print(f"***\t\tsample [{r}, {c}]\t\t***")
            input = generateInput(N, r*c)
            t=xpuzzle.XPuzzle(r, c, input[0]) 
            s=search.Search(t, 120)

            p  = s.aStar(1)
            print(f"{p[0]} time: {p[2]}")
            if isinstance(p[0], node.Node):
                star1_stat.append([r, c, p[2]])
            else:
                star1_stat.append([r, c, -1])
    
    return star1_stat

#main
def process(input):
    for index, e in enumerate(input):
        print(f"***\t\tsample {index}\t\t***")
        t=xpuzzle.XPuzzle(2, 4, input[index]) 
        s=search.Search(t)

        p = s.uniformCost()
        print(p[0])
        exportData("ucs", index, p)

        for heuristic in range(2):
            p = s.greedyBFS(heuristic)
            print(p[0])
            exportData("gbfs", index, p, heuristic)

            p  = s.aStar(heuristic)
            print(p[0])
            exportData("astar", index, p, heuristic)

In [5]:
inputData = importData('samplePuzzles.txt')
print(inputData, end='\n\n')
process(inputData)

[[3 0 1 4 2 6 5 7]
 [6 3 4 7 1 2 5 0]
 [1 0 3 6 5 2 7 4]
 [0 3 5 1 7 2 4 6]
 [1 3 5 7 0 4 6 2]]

***		sample 0		***
LDCLDLLUZD
RDLULZDTRRRTUCLLLZ
LDCLDLLUZD
RDLLCDTRULDRRURDLULLDZ
LDCLDLLUZD
***		sample 1		***
LURTDRRRULDR
CDRURDRULLDLCLLDRR
LURTDRRRULDR
CDRURDRULLDLCLLDRR
LURTDRRRULDR
***		sample 2		***
RRDLULDRR
RRDLULDRR
RRDLULDRR
RRDLULDRR
RRDLULDRR
***		sample 3		***
ZCRRR
ZCRRR
ZCRRR
ZCRRR
ZCRRR
***		sample 4		***
Z
Z
Z
Z
Z


In [164]:
inputData = importData('samplePuzzles.txt')
print(inputData, end='\n\n')
#inputData[4] no solution
index = 4
t=xpuzzle.XPuzzle(2, 4, inputData[index]) 
t.printPuzzle()
s=search.Search(t)

[[3 0 1 4 2 6 5 7]
 [6 3 4 7 1 2 5 0]
 [1 0 3 6 5 2 7 4]
 [0 3 5 1 7 2 4 6]
 [1 3 5 7 0 4 6 1]]

1 3 5 7 
0 4 6 1 



In [165]:
print("A*")
for heuristic in range(2):
    p  = s.aStar(heuristic)
    print(p[0])
    exportData("astar", index, p, heuristic)


A*
no solution
no solution


In [166]:
print("GBFS")
for heuristic in range(2):
    p = s.greedyBFS(heuristic)
    print(p[0])
    exportData("gbfs", index, p, heuristic)


GBFS
no solution
no solution


In [167]:
print("UCS")
p = s.uniformCost()
print(p[0])
exportData("ucs", index, p)

print("\nMoves")
move = str(p[0])
t.parseMoveSequence(move)


UCS
no solution

Moves
1 3 5 7 
0 4 6 1 

1 3 5 7 
0 4 6 1 

1 3 5 7 
0 4 6 1 

1 3 5 7 
0 4 6 1 

1 3 5 7 
0 4 6 1 

1 3 5 7 
0 4 6 1 

1 3 5 7 
0 4 6 1 

1 3 5 7 
0 4 6 1 

1 3 5 7 
0 4 6 1 

1 3 5 7 
0 4 6 1 

1 3 5 7 
0 4 6 1 



In [12]:
stat1 = analysis()
print(stat1)

***		sample 0		***
CZLLLURRDR
***		sample 1		***
CLLLURDRURD
***		sample 2		***
no solution
***		sample 3		***
no solution
***		sample 4		***
DRURDCDRRR
***		sample 5		***
no solution
***		sample 6		***
URDLLLURDLZ
***		sample 7		***
RULDLLURDLCD
***		sample 8		***
LLURRDTRRR
***		sample 9		***
no solution
***		sample 10		***
UZLDLULDCD
***		sample 11		***
LDRRRTRULDZ
***		sample 12		***
RTCLDLLZ
***		sample 13		***
no solution
***		sample 14		***
no solution
***		sample 15		***
no solution
***		sample 16		***
DLURTRRDLLZ
***		sample 17		***
LDLLURRRD
***		sample 18		***
no solution
***		sample 19		***
LDRULLDRULC
***		sample 20		***
no solution
***		sample 21		***
no solution
***		sample 22		***
DRUTDRURDR
***		sample 23		***
LZDLULLZD
***		sample 24		***
no solution
***		sample 25		***
no solution
***		sample 26		***
DLUCLULLC
***		sample 27		***
no solution
***		sample 28		***
no solution
***		sample 29		***
no solution
***		sample 30		***
no solution
***		sample 31		***
DLURRRTRRDR

In [21]:
stat2 = scaleup(6, 8)
print(stat2)

***		sample [2, 2]		***
URD time: 0.000993490219116211
***		sample [2, 3]		***
RDLUCLLZ time: 0.041110992431640625
***		sample [2, 4]		***
DRRTUZLLDRURD time: 0.04199981689453125
***		sample [2, 5]		***
no solution time: 120
***		sample [2, 6]		***
no solution time: 120
***		sample [2, 7]		***
no solution time: 120
***		sample [2, 8]		***
no solution time: 120
***		sample [2, 9]		***


KeyboardInterrupt: 